In [1]:
try:
  %tensorflow_version 1.x
except:
  pass

import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.ensemble import RandomForestClassifier

from sklearn.decomposition import TruncatedSVD

Using TensorFlow backend.


In [2]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
except:
  print("You are not running on Google Colab. Remember to download the data and update the paths accordingly.")
  pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
basepath = '/content/drive/My Drive/Text Mining & Search PROJECT/'

## Random forest with Singular Value Decomposition

In [0]:
tfidf_mat_svd, y = joblib.load(os.path.join(basepath, 'tfidf_matrix_svd.joblib'))

In [6]:
tfidf_mat_svd.shape

(500000, 500)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_mat_svd, y, 
                                                    test_size = 0.1,
                                                    random_state=1)

In [0]:
clf = RandomForestClassifier(n_estimators=40, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

In [0]:
rf_score=clf.score(X_test,y_test)

0.85502

## Neural Network

In [0]:
train_vect, y = joblib.load(os.path.join(basepath, 'tfidf_matrix_5class.joblib'))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_vect, y, test_size=0.1, 
                                                    random_state=1)

In [0]:
n_dim = X_train.shape[1]
n_classes = 5

In [0]:
def build_model():
  model = Sequential()
  # layer 1
  model.add(Dense(1024, input_dim=n_dim, activation='relu'))
  model.add(Dropout(rate=0.5))
  # layer 2
  model.add(Dense(512, activation='relu'))
  # dropout
  model.add(Dropout(rate=0.2))
  # layer 3
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(rate=0.2))
  # layer 4
  model.add(Dense(32, activation='relu'))
  # output layer
  model.add(Dense(n_classes, activation='softmax'))
  
  model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', 
                metrics=['sparse_categorical_accuracy'])
  
  return model

In [0]:
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

In [14]:
model = build_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1024)              10241024  
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)               

In [0]:
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    callbacks=[earlystop],
                    epochs=50,
                    batch_size=128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 450000 samples, validate on 50000 samples
Epoch 1/50





450000/450000 [==============================] - 52s 115us/step - loss: 0.2050 - acc: 0.9327 - val_loss: 0.1626 - val_acc: 0.9469
Epoch 2/50
450000/450000 [==============================] - 43s 96us/step - loss: 0.1314 - acc: 0.9571 - val_loss: 0.1521 - val_acc: 0.9503
Epoch 3/50
450000/450000 [==============================] - 43s 96us/step - loss: 0.0863 - acc: 0.9719 - val_loss: 0.1636 - val_acc: 0.9500
Epoch 4/50
450000/450000 [==============================] - 43s 96us/step - loss: 0.0537 - acc: 0.9826 - val_loss: 0.1819 - val_acc: 0.9496
Epoch 5/50
450000/450000 [==============================] - 43s 96us/step - loss: 0.0355 - acc: 0.9885 - val_loss: 0.2051 - val_acc: 0.9495
Epoch 00005: early stopping


In [0]:
model.evaluate(X_test,y_test_cat)

50000/50000 [==============================] - 4s 84us/step


[0.2051233537350921, 0.94954]